## Импорт библиотек

Загружаются основные библиотеки для анализа химических структур:

- `rdkit`: для обработки SMILES и построения молекул;
- `descriptastorus`: генерация дескрипторов RDKit2D;
- `mordred`: расширенная генерация химико-информатических дескрипторов;
- `pandas`/`numpy`: для работы с табличными и числовыми данными.

Импортируются генераторы дескрипторов, включая `MakeGenerator`, с возможностью получения предобученных дескрипторных векторов.

In [91]:
import pandas as pd
import numpy as np

from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import PandasTools
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import AllChem
from rdkit.Chem import rdFingerprintGenerator
from rdkit import DataStructs
from rdkit.Chem import MACCSkeys
from rdkit.Chem.Fingerprints import FingerprintMols
from descriptastorus.descriptors.DescriptorGenerator import MakeGenerator
from mordred import Calculator, descriptors
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator

## Загрузка данных

Загружаются обучающая и тестовая выборки. В качестве индекса используется идентификатор CASRN. 

In [92]:
train = pd.read_csv('data/row/train.csv', index_col = 'CASRN')
test = pd.read_csv('data/row/test.csv', index_col = 'CASRN')

## Конвертация SMILES в молекулы
С помощью RDKit SMILES-строки преобразуются в молекулярные объекты (ROMol). Это необходимо для извлечения структурных признаков.

In [93]:
PandasTools.AddMoleculeColumnToFrame(train,smilesCol='SMILES')
PandasTools.AddMoleculeColumnToFrame(test,smilesCol='SMILES')

# ECFP6 Bits

#### Генерация ECFP6 с заданными параметрами (радиус 3, размер 2048 бит)

Используется объект `GetMorganGenerator` из RDKit для создания фингерпринтов ECFP6:

- `radius=3` — радиус охвата атомов (ECFP6);
- `fpSize=2048` — размер выходного бинарного вектора.



In [94]:
generator = GetMorganGenerator(radius=3, fpSize=2048)

In [95]:
train_ECFP6 = [generator.GetFingerprint(x) for x in train['ROMol']]
test_ECFP6 = [generator.GetFingerprint(x) for x in test['ROMol']]

#### Преобразование ECFP6-фингерпринтов в списки

Преобразуются RDKit-фингерпринты в обычные списки Python (поэлементно: 0/1).  
Это необходимо для последующей упаковки в датафреймы или массивы `NumPy`, чтобы использовать их в качестве входных данных в модели машинного обучения.

In [96]:
train_ecfp6_lists = [list(l) for l in train_ECFP6]
test_ecfp6_lists = [list(l) for l in test_ECFP6]

Генерируем список строк `Bit_0`, `Bit_1`, ..., `Bit_2047`.


In [97]:
ecfp6_name = [f'Bit_{i}' for i in range(2048)]

Создаются DataFrame для тренировочных и тестовых данных, где строки индексированы по индексам `train` и `test`, а столбцы названы в соответствии с `ecfp6_name`.


In [98]:
train_ecfp6_df = pd.DataFrame(train_ecfp6_lists, index = train.index, columns=ecfp6_name)
test_ecfp6_df = pd.DataFrame(test_ecfp6_lists, index = test.index, columns=ecfp6_name)

Вывод размеров DataFrame для тренировочных и тестовых данных: `train_ecfp6_df` и `test_ecfp6_df`.


In [99]:
train_ecfp6_df.shape, test_ecfp6_df.shape


((8221, 2048), (2849, 2048))

Сохранение DataFrame с тренировочными и тестовыми данными в CSV файлы: `train_ecfp6_bits.csv` и `test_ecfp6_bits.csv`.


In [100]:
train_ecfp6_df.to_csv('data/descriptors/train_ecfp6_bits.csv')
test_ecfp6_df.to_csv('data/descriptors/test_ecfp6_bits.csv')

#### Далее аналогично

## ECFP6 counts

Создание генератора с подсчётом частот (counts), радиус = 3, размер вектора = 2048

In [101]:
generator = GetMorganGenerator(radius=3, fpSize=2048, countSimulation=True)

Генерация фингерпринтов для train и test

In [102]:
train_ECFP6_counts = [generator.GetFingerprint(x) for x in train['ROMol']]
test_ECFP6_counts = [generator.GetFingerprint(x) for x in test['ROMol']]

Преобразование объектов типа RDKit ExplicitBitVect в списки

In [103]:
train_ecfp6_counts_lists = [list(l) for l in train_ECFP6_counts]
test_ecfp6__counts_lists = [list(l) for l in test_ECFP6_counts]

Создание DataFrame

In [104]:
train_ecfp6_counts_df = pd.DataFrame(train_ecfp6_counts_lists, index = train.index, columns=ecfp6_name)
test_ecfp6_counts_df = pd.DataFrame(test_ecfp6__counts_lists, index = test.index, columns=ecfp6_name)

In [105]:
train_ecfp6_counts_df.shape, test_ecfp6_counts_df.shape

((8221, 2048), (2849, 2048))

In [106]:
train_ecfp6_counts_df.to_csv('data/descriptors/train_ecfp6_counts.csv')
test_ecfp6_counts_df.to_csv('data/descriptors/test_ecfp6_counts.csv')

## MACCS keys

#### Генерация MACCS ключей для молекул

Генерация MACCS fingerprints для обучающей и тестовой выборок

In [107]:
# MACCS keys
train_maccs = [MACCSkeys.GenMACCSKeys(x) for x in train['ROMol']]
test_maccs = [MACCSkeys.GenMACCSKeys(x) for x in test['ROMol']]

 Преобразуем RDKit объекты фингерпринтов в списки битов (0 или 1)

In [108]:
train_maccs_lists = [list(l) for l in train_maccs]
test_maccs_lists = [list(l) for l in test_maccs]

Генерируем имена колонок: MACCS включает 167 битов (от 0 до 166)

In [109]:
maccs_name = [f'Bit_{i}' for i in range(167)]

Создаём датафреймы с индексами как в оригинальных данных и названиями колонок

In [110]:
train_maccs_df = pd.DataFrame(train_maccs_lists, index = train.index, columns=maccs_name)
test_maccs_df = pd.DataFrame(test_maccs_lists, index = test.index, columns=maccs_name)

Проверка размеров полученных датафреймов

In [111]:
train_maccs_df.shape, test_maccs_df.shape


((8221, 167), (2849, 167))

Сохранение MACCS фингерпринтов в CSV

In [112]:
train_maccs_df.to_csv('data/descriptors/train_maccs.csv')
test_maccs_df.to_csv('data/descriptors/test_maccs.csv')

## RDKit

Инициализация генератора дескрипторов RDKit2D

generator = MakeGenerator(("RDKit2D",))

Генерация RDKit2D дескрипторов с помощью MakeGenerator

In [ ]:
train_rdkit2d = [generator.process(x)[1:] for x in train['SMILES']]
test_rdkit2d = [generator.process(x)[1:] for x in test['SMILES']]

Получаем имена всех дескрипторов (начиная со второго — пропускаем ID)

In [ ]:
rdkit2d_name = []
for name, numpy_type in generator.GetColumns():
    rdkit2d_name.append(name)

Создание датафреймов RDKit2D дескрипторов

In [115]:
train_rdkit2d_df = pd.DataFrame(train_rdkit2d, index = train.index, columns=rdkit2d_name[1:])
test_rdkit2d_df = pd.DataFrame(test_rdkit2d, index = test.index, columns=rdkit2d_name[1:])

Проверка размеров датафреймов

In [116]:
train_rdkit2d_df.shape, test_rdkit2d_df.shape

((8221, 200), (2849, 200))

Сохранение RDKit2D дескрипторов в .csv

In [117]:
train_rdkit2d_df.to_csv('data/descriptors/train_rdkit2d.csv')
test_rdkit2d_df.to_csv('data/descriptors/test_rdkit2d.csv')

## Mordred

Создаем объект калькулятора молекулярных дескрипторов Mordred, игнорируя 3D-данные (так как они не нужны без SDF или MOL файла)

In [ ]:
mordred_calc = Calculator(descriptors, ignore_3D=True) 

Применяем калькулятор к наборам молекул для обучающей и тестовой выборок и сохраняем результат в DataFrame

In [ ]:
train_mordred = mordred_calc.pandas([mol for mol in train['ROMol']])
test_mordred = mordred_calc.pandas([mol for mol in test['ROMol']])

  2%|▏         | 125/8221 [00:10<10:34, 12.76it/s]

c:\Users\PC\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
c:\Users\PC\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


  2%|▏         | 171/8221 [00:13<07:29, 17.91it/s]

c:\Users\PC\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


  3%|▎         | 236/8221 [00:16<06:03, 21.95it/s]

...

c:\Users\PC\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


  7%|▋         | 614/8221 [00:39<06:32, 19.38it/s]

c:\Users\PC\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
c:\Users\PC\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
c:\Users\PC\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
c:\Users\PC\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
c:\Users\PC\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
c:\Users\PC\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  r

 11%|█         | 890/8221 [00:53<07:22, 16.57it/s]


KeyboardInterrupt: 

Проверка размеров датафреймов

In [ ]:
train_mordred.shape, test_mordred.shape

Удаляем ненумерические признаки (оставляем только числовые типы данных)

In [ ]:
train_mordred = train_mordred.select_dtypes(include=['float64', 'int64', 'float'])

Приводим тестовый набор данных к тем же признакам, что и в обучающем наборе

In [ ]:
test_mordred = test_mordred[list(train_mordred)]

Выводим размеры обучающего и тестового наборов данных

In [ ]:
train_mordred.shape, test_mordred.shape

Выводим первые строки тестового набора данных для проверки его структуры

In [ ]:
test_mordred.head(1)

Сравниваем список признаков в обучающем и тестовом наборах данных, чтобы убедиться, что они одинаковы

In [ ]:
list(train_mordred) == list(test_mordred)

Устанавливаем индексы для обучающего и тестового наборов данных, чтобы они совпадали с исходными индексами

In [ ]:
train_mordred.index = train.index
test_mordred.index = test.index

Выводим размеры обучающего и тестового наборов данных

In [ ]:
train_mordred.shape, test_mordred.shape


Сохраняем обучающий и тестовый наборы данных с дескрипторами в CSV файлы для дальнейшего использования

In [ ]:
train_mordred.to_csv('data/descriptors/train_mordred.csv')
test_mordred.to_csv('data/descriptors/test_mordred.csv')